In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 21341307
paper_name = 'yoshikawa_shimizu_2011' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/yea_1843_supportinginforTS1.xlsx', sheet_name='Deletion', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4727 x 11


In [10]:
original_data.head()

,Unnamed: 0,Name,Name.1,"0: Slow, 1:no-change, 2:High",Average,1st experiment,2nd experiment,Parental starin,Unnamed: 8,Unnamed: 9,Unnamed: 10
0,1,YAL068C,YAL068C,1,0.355198,0.357846,0.352549,BY4739,NaN,NaN,NaN
1,2,YAL067C,SEO1,1,0.355486,0.352233,0.358740,BY4739,NaN,NaN,NaN
2,3,YAL066W,YAL066W,1,0.350824,0.351376,0.350273,BY4739,NaN,NaN,NaN
3,4,YAL065C,YAL065C,1,0.327704,0.335002,0.320407,BY4739,NaN,NaN,NaN
4,5,YAL062W,GDH3,1,0.354422,0.356379,0.352466,BY4739,NaN,NaN,NaN


In [11]:
original_data['orf'] = original_data['Name'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Unnamed: 0, Name, Name.1, 0: Slow, 1:no-change, 2:High, Average, 1st experiment, 2nd experiment, Parental starin, Unnamed: 8, Unnamed: 9, Unnamed: 10, orf]
Index: []


In [15]:
original_data['data'] = original_data['Average'].astype(float)

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(4703, 1)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [415]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,415
data_type,value
orf,
YAL002W,0.336660
YAL004W,0.335833
YAL005C,0.348510
YAL007C,0.309204
YAL008W,0.335025


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 26


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,415
,data_type,value
gene_id,orf,
2,YAL002W,0.336660
1863,YAL004W,0.335833
4,YAL005C,0.348510
5,YAL007C,0.309204
6,YAL008W,0.335025


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id            415          
data_type           value    valuez
gene_id orf                        
2       YAL002W  0.336660  0.061377
1863    YAL004W  0.335833  0.036698
4       YAL005C  0.348510  0.415255
5       YAL007C  0.309204 -0.758522
6       YAL008W  0.335025  0.012554

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [30]:
from IO.save_data_to_db3 import *

In [31]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 21341307...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.96s/it]

Updating the data_modified_on field...
